In [2]:
import statsapi
import pandas as pd

In [2]:
# Test pulling info for Ohtani
player_info = statsapi.get('person', {'personId': 660271})
print(player_info)

{'copyright': 'Copyright 2025 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt', 'people': [{'id': 660271, 'fullName': 'Shohei Ohtani', 'link': '/api/v1/people/660271', 'firstName': 'Shohei', 'lastName': 'Ohtani', 'primaryNumber': '17', 'birthDate': '1994-07-05', 'currentAge': 30, 'birthCity': 'Oshu', 'birthCountry': 'Japan', 'height': '6\' 3"', 'weight': 210, 'active': True, 'primaryPosition': {'code': 'Y', 'name': 'Two-Way Player', 'type': 'Two-Way Player', 'abbreviation': 'TWP'}, 'useName': 'Shohei', 'useLastName': 'Ohtani', 'boxscoreName': 'Ohtani', 'nickName': 'Showtime', 'gender': 'M', 'isPlayer': True, 'isVerified': False, 'pronunciation': 'show-HEY oh-TAWN-ee', 'mlbDebutDate': '2018-03-29', 'batSide': {'code': 'L', 'description': 'Left'}, 'pitchHand': {'code': 'R', 'description': 'Right'}, 'nameFirstLast': 'Shohei Ohtani', 'nameSlug': 'shohei-ohtani-660271', 'firstLastName': 'Sh

In [8]:
# DONT RUN THIS IT TAKES A WHILE
# def get_team_ids():
#     teams = statsapi.get('teams', {'sportId': 1})['teams']
#     return [team['id'] for team in teams]

# def get_players_from_team(team_id, season='2021'):
#     roster = statsapi.get('team_roster', {'teamId': team_id, 'season': season})
#     return roster['roster']

# def get_player_info_flat(player_id):
#     person = statsapi.get('person', {'personId': player_id})
#     info = person['people'][0]

#     return {
#         'id': info.get('id'),
#         'fullName': info.get('fullName'),
#         'birthDate': info.get('birthDate'),
#         'birthCountry': info.get('birthCountry'),
#         'height': info.get('height'),
#         'weight': info.get('weight'),
#         'primaryPosition': info.get('primaryPosition', {}).get('abbreviation'),
#         'batSide': info.get('batSide', {}).get('code'),
#         'pitchHand': info.get('pitchHand', {}).get('code'),
#         'debutDate': info.get('mlbDebutDate'),
#         'active': info.get('active'),
#     }

# def get_all_players_df(season='2021'):
#     team_ids = get_team_ids()
#     player_data = []

#     for team_id in team_ids:
#         players = get_players_from_team(team_id, season)
#         for player in players:
#             player_id = player['person']['id']
#             info = get_player_info_flat(player_id)
#             player_data.append(info)

#     df = pd.DataFrame(player_data)
#     return df

# df_players = get_all_players_df()
# print(df_players.head())


Fetching team 133...
Fetching team 134...
Fetching team 135...
Fetching team 136...
Fetching team 137...
Fetching team 138...
Fetching team 139...
Fetching team 140...
Fetching team 141...
Fetching team 142...
Fetching team 143...
Fetching team 144...
Fetching team 145...
Fetching team 146...
Fetching team 147...
Fetching team 158...
Fetching team 108...
Fetching team 109...
Fetching team 110...
Fetching team 111...
Fetching team 112...
Fetching team 113...
Fetching team 114...
Fetching team 115...
Fetching team 116...
Fetching team 117...
Fetching team 118...
Fetching team 119...
Fetching team 120...
Fetching team 121...
       id         fullName   birthDate        birthCountry height  weight  \
0  642758  Domingo Acevedo  1994-03-06  Dominican Republic  6' 7"     240   
1  664119     Austin Allen  1994-01-16                 USA  6' 1"     219   
2  462101     Elvis Andrus  1988-08-26           Venezuela  6' 0"     210   
3  642456     Luis Barrera  1995-11-15  Dominican Republic  6'

In [9]:
# df_players.head()

,id,fullName,birthDate,birthCountry,height,weight,primaryPosition,batSide,pitchHand,debutDate,active
0,642758,Domingo Acevedo,1994-03-06,Dominican Republic,"6' 7""",240,P,R,R,2021-06-21,True
1,664119,Austin Allen,1994-01-16,USA,"6' 1""",219,C,L,R,2019-05-11,False
2,462101,Elvis Andrus,1988-08-26,Venezuela,"6' 0""",210,SS,R,R,2009-04-06,False
3,642456,Luis Barrera,1995-11-15,Dominican Republic,"6' 0""",195,OF,L,L,2021-05-19,True
4,605135,Chris Bassitt,1989-02-22,USA,"6' 5""",217,P,R,R,2014-08-30,True


In [10]:
# df_players.to_csv('mlb_players_2021.csv', index=False)

### **Only run the code below here because everyone should have the two dfs on their computer !**

In [3]:
players = pd.read_csv("mlb_players_2021.csv")
pitches = pd.read_csv("regseason.csv")

In [4]:
players.rename(columns={"id" : "batter","fullName" : "batter_fullName" , "birthCountry" : "batter_birthCountry"}, inplace=True)
players.head()

,batter,batter_fullName,birthDate,batter_birthCountry,height,weight,primaryPosition,batSide,pitchHand,debutDate,active
0,642758,Domingo Acevedo,1994-03-06,Dominican Republic,"6' 7""",240,P,R,R,2021-06-21,True
1,664119,Austin Allen,1994-01-16,USA,"6' 1""",219,C,L,R,2019-05-11,False
2,462101,Elvis Andrus,1988-08-26,Venezuela,"6' 0""",210,SS,R,R,2009-04-06,False
3,642456,Luis Barrera,1995-11-15,Dominican Republic,"6' 0""",195,OF,L,L,2021-05-19,True
4,605135,Chris Bassitt,1989-02-22,USA,"6' 5""",217,P,R,R,2014-08-30,True


In [10]:
pitches.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,count,PBhand,pitch_cat
0,1,SL,2021-10-03,85.1,2.39,6.05,"Detmers, Reid",641487,672282,single,...,2,4,Standard,Standard,293.0,0.079,0.780,0_1,bLpL,breaking
1,2,FF,2021-10-03,93.8,2.31,6.03,"Detmers, Reid",641487,672282,NaN,...,1,4,Standard,Standard,150.0,0.000,-0.045,0_0,bLpL,fastball
2,3,CU,2021-10-03,73.9,2.12,6.25,"Detmers, Reid",664238,672282,strikeout,...,1,4,Standard,Standard,329.0,-0.032,-0.315,2_2,bRpL,breaking
3,4,FF,2021-10-03,94.6,2.43,5.96,"Detmers, Reid",664238,672282,NaN,...,1,4,Standard,Standard,146.0,0.000,0.047,1_2,bRpL,fastball
4,5,FF,2021-10-03,94.3,2.08,6.15,"Detmers, Reid",664238,672282,NaN,...,1,4,Standard,Standard,151.0,0.000,0.000,1_2,bRpL,fastball


In [5]:
# Merge pitches df with info about the batter
full_df = pd.merge(pitches, players[["batter","batter_fullName","batSide", "batter_birthCountry"]], on="batter")
full_df.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,count,PBhand,pitch_cat,batter_fullName,batSide,batter_birthCountry
0,1,SL,2021-10-03,85.1,2.39,6.05,"Detmers, Reid",641487,672282,single,...,Standard,293.0,0.079,0.780,0_1,bLpL,breaking,J.P. Crawford,L,USA
1,2,FF,2021-10-03,93.8,2.31,6.03,"Detmers, Reid",641487,672282,NaN,...,Standard,150.0,0.000,-0.045,0_0,bLpL,fastball,J.P. Crawford,L,USA
2,3,CU,2021-10-03,73.9,2.12,6.25,"Detmers, Reid",664238,672282,strikeout,...,Standard,329.0,-0.032,-0.315,2_2,bRpL,breaking,Dylan Moore,R,USA
3,4,FF,2021-10-03,94.6,2.43,5.96,"Detmers, Reid",664238,672282,NaN,...,Standard,146.0,0.000,0.047,1_2,bRpL,fastball,Dylan Moore,R,USA
4,5,FF,2021-10-03,94.3,2.08,6.15,"Detmers, Reid",664238,672282,NaN,...,Standard,151.0,0.000,0.000,1_2,bRpL,fastball,Dylan Moore,R,USA


In [41]:
# Remove the pitches that were fouled, in play, or swung at
not_hit = ["called_strike", "ball"]
pitches_not_hit = full_df[full_df['description'].isin(not_hit)].copy()
len(pitches_not_hit)

390288

So we have 390,000 pitches to work with that were thrown but not swung at or hit

In [43]:
# Check that we only have balls and called strikes
pitches_not_hit['description'].unique()

array(['called_strike', 'ball'], dtype=object)

In [52]:
pitches_not_hit["strike_zone"] = pitches_not_hit["zone"] <= 9 # Determine if the pitch was actually in the zone

pitches_not_hit["called_correctly"] = (
    ((pitches_not_hit["zone"] <= 9) & (pitches_not_hit["type"] == "S")) | # Pitch is in the zone and is called a strike  
    ((pitches_not_hit["zone"] > 9) & (pitches_not_hit["type"] == "B"))    # Pitch is not in the zone and is called a ball
) 
pitches_not_hit["called_correctly"] = pitches_not_hit["called_correctly"].astype(int) # This will be are target variable -- whether the pitch was called "correctly"

In [53]:
# Check how many pitches we have that were "called correctly"
len(pitches_not_hit[pitches_not_hit["called_correctly"] == 1])

357779

In [54]:
# Check how many pitches we have that were not "called correctly"
len(pitches_not_hit[pitches_not_hit["called_correctly"] == 0])

32509

So there are aound 360,000 correctly called pitches at 32,000 which are not called correctly -- we might want to consider removing some of the correctly called pitches so that we have the same amount for each. But 64,000 data points should be enough for training our model. 